In [2]:
import numpy as np
import pandas as pd
import folium
from haversine import haversine

### 만들어 둔 CSV 파일 불러오기

In [7]:
total_df = pd.read_csv("total.csv", encoding = 'ms949')
total_df.drop('Unnamed: 0', axis=1, inplace=True)


load_df = pd.read_csv("total_load_points.csv", encoding = 'ms949')
load_df.drop('Unnamed: 0', axis=1, inplace=True)
load_df.head(5)

total_df.head(5)


,분류,명칭,위도,경도
0,보안등,역삼1동833,37.493349,127.032033
1,보안등,역삼1동833,37.493349,127.032033
2,보안등,역삼1동833,37.493349,127.032033
3,보안등,역삼1동833,37.493349,127.032033
4,보안등,역삼1동833,37.493349,127.032033


### 모든 데이터들을 담을 데이터 프레임을 만든다.

밝기 = 각 시설물의 점수      EX) 파출소 = 5점, 보안등 = 4점, 편의점 = 3점, cctv = 2점
총밝기 = '기준 시설물의 점수' + '주변 시설물의 점수'

In [64]:
all_df = pd.DataFrame(columns=['point_ID', '거리', '밝기', '총밝기', '분류', '명칭', '위도', '경도', '경유여부'])


### 밝기 점수 주기

for i in range(len(total_df)) :
    score = 0 
    if total_df['분류'][i] == '파출소' :
        score += 5
    elif total_df['분류'][i] == '보안등' :    
        score += 4
    elif total_df['분류'][i] == '편의점' :    
        score += 3
    else :    
        score += 2
        
    tmp = {'point_ID' : 'Point_%d' % i, '거리' : 0, 
           '밝기' : score, '경유여부' : 0, 
           '분류' :total_df.loc[i,'분류'], '명칭' : total_df.loc[i,'명칭'], 
           '위도' : total_df.loc[i,'위도'], '경도' : total_df.loc[i,'경도'] 
           } 
    all_df = all_df.append(all_df.from_dict(tmp, orient = 'index').T)

    
### 데이터 프레임 인덱스 정리 및, 컬럼명 정리    

all_df = all_df.reset_index(drop=True)
all_df = pd.DataFrame(all_df, columns = ['point_ID','거리', '밝기', '총밝기', '분류', '명칭', '위도', '경도', '경유여부'])


### 총밝기 구하기    

scores = []
for i in range(len(all_df)) :
    score = 0
    s_point = (all_df.loc[i,'위도'], all_df.loc[i,'경도']) # 시작점
    for n in range(len(all_df)) :
        t_point = (all_df.loc[n,'위도'], all_df.loc[n,'경도']) # 타겟
        d_m = haversine(s_point, t_point, unit='m') #  시작점과 타겟의 거리  단위 미터
        # 반경 50M 를 범위로 잡음
        if d_m <= 50 :
            score += all_df.loc[n, '밝기']
    # print(i, score) # 잘나옴
    scores += [score]
    


C:\Python3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [65]:
all_df['총밝기'] = pd.DataFrame(scores, columns = ['총밝기'])
all_df

,point_ID,거리,밝기,총밝기,분류,명칭,위도,경도,경유여부
0,Point_0,0,4,110,보안등,역삼1동833,37.4933,127.032,0
1,Point_1,0,4,110,보안등,역삼1동833,37.4933,127.032,0
2,Point_2,0,4,110,보안등,역삼1동833,37.4933,127.032,0
3,Point_3,0,4,110,보안등,역삼1동833,37.4933,127.032,0
4,Point_4,0,4,110,보안등,역삼1동833,37.4933,127.032,0
5,Point_5,0,4,16,보안등,역삼1동832-25,37.4928,127.031,0
6,Point_6,0,4,20,보안등,역삼1동832-27,37.4926,127.032,0
7,Point_7,0,4,23,보안등,역삼1동832-28,37.4925,127.032,0
8,Point_8,0,4,23,보안등,역삼1동832-28,37.4925,127.032,0
9,Point_9,0,4,27,보안등,역삼1동832-29,37.4923,127.032,0


In [66]:
# DataFrame 을 csv 파일로 저장하기

all_df.to_csv("all_df.csv", mode='w', encoding='ms949')   # encoding 을 안 하면 한글이 깨짐